In [1]:
import pandas as pd

In [3]:
df_gene = pd.read_csv('IMGT_GENEs.csv', index_col=0)

In [4]:
df_gene.head()

,accession number,name,species,functionality,exon/name/label,start end positions,number of nucleotides,codon start,+n,+/-n,error corrections,number of AA,number of char,partial,reverse complementary,FASTA
0,KT723008,IGHA*01,Bos taurus_Holstein,F,CH1,"n,652007..652311",306 nt,1,+1,-1,,102 AA,102+0=102,,,XSETSPSIFPLSLGNNDPAGQVVIGCLVQGFFPSAPLSVTWNQNGD...
1,KT723008,IGHA*01,Bos taurus_Holstein,F,H-CH2,"g,652493..652821",330 nt,1,+1,-1,,110 AA,110+0=110,,,DSSSCCVPNCEPSLSVQPPALEDLLLGSNASLTCTLSGLKSAEGAS...
2,KT723008,IGHA*01,Bos taurus_Holstein,F,CH3-CHS,"g,653011..653402",393 nt,1,+1,,,131 AA,131+0=131,,,GNTFRPQVHLLPPPSEELALNELVTLTCLVRGFSPKEVLVRWLQGN...
3,KT723008,IGHA*01,Bos taurus_Holstein,F,M,"g,655709..655893",186 nt,1,+1,,,62 AA,62+0=62,,,EHQPWLVLDLMQSSPEEDSPEASLWPTTVTLLTLFLLSLFYSTALT...
4,KT723008,IGHD*02,Bos taurus_Holstein,ORF,CH1,"n,525572..525894",324 nt,1,+1,-1,,108 AA,108+0=108,,,XGESHLRVFSLVSCVSSPSDESTVALGCLARDFVPNSVSFSWKFNN...


In [6]:
def longest_common_substring(s1, s2):
    m, n = len(s1), len(s2)
    # Create a 2D array to store lengths of longest common suffixes
    lcsuff = [[0] * (n + 1) for _ in range(m + 1)]
    length = 0  # Length of longest common substring
    end_pos = 0  # Ending position of longest common substring in s1

    # Build the lcsuff array
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i - 1] == s2[j - 1]:
                lcsuff[i][j] = lcsuff[i - 1][j - 1] + 1
                if lcsuff[i][j] > length:
                    length = lcsuff[i][j]
                    end_pos = i
            else:
                lcsuff[i][j] = 0

    # The longest common substring
    return s1[end_pos - length: end_pos]

In [19]:
# search for the gene based on name and species from IMGT_GENE_DB
def find_sequence(dataframe, name, species='Homo sapiens'):
    found = dataframe.loc[(dataframe['name'] == name) & (dataframe['species'] == species)]
    if len(found) > 0:
        return found['FASTA'].iloc[0]
    return ""

In [5]:
v_b = "TRBV7-9*01"
j_b = "TRBJ1-5*01"
cdr3 = "CASSLAVGEGPQHF"

In [14]:
v_seq = find_sequence(df_gene, v_b)
j_seq = find_sequence(df_gene, j_b)

In [23]:
#get the full sequence from finding the longest common string in both v and j gene sequences
#return the replaced full sequence and the begin and end AA number(index)
def get_tcr_seq_cdr_region(gene_dataframe, v_gene, j_gene, cdr3, species="Homo sapiens"):
    v_seq = find_sequence(gene_dataframe, v_gene, species)
    j_seq = find_sequence(gene_dataframe, j_gene, species)
    #find the longest common string beween gene and cdr3
    v_common = longest_common_substring(cdr3, v_seq)
    j_common = longest_common_substring(cdr3, j_seq)
    if len(v_common) == 0 or len(j_common) == 0:
        raise AssertionError("Cannot find matching part")
    full_seq = ""
    begin = v_seq.index(v_common)
    end = begin + len(cdr3)
    end_j = j_seq.index(j_common) + len(j_common)
    #assemble v gene (until the beginning of cdr3) + cdr3 + j gene (untile the end of cdr3)
    full_seq = v_seq[:begin] + cdr3 + j_seq[end_j:]
    return begin, end, full_seq

In [15]:
a = longest_common_substring(cdr3, v_seq)
print(a)

CASSL


In [16]:
b = longest_common_substring(cdr3, j_seq)
print(b)

PQHF


In [24]:
begin, end, full_seq = get_tcr_seq_cdr_region(df_gene, v_b, j_b, cdr3)

In [25]:
full_seq[begin: end]

'CASSLAVGEGPQHF'

In [26]:
print(full_seq)
print(j_seq)

DTGVSQNPRHKITKRGQNVTFRCDPISEHNRLYWYRQTLGQGPEFLTYFQNEAQLEKSRLLSDRFSAERPKGSFSTLEIQRTEQGDSAMYLCASSLAVGEGPQHFGDGTRLSIL
SNQPQHFGDGTRLSIL
